In [1]:
import numpy as np
import glob
import cv2
import blobfile as bf
import PIL
import copy
from pytorch_lightning import seed_everything
seed_everything(47)

def _list_image_files_recursively(data_dir):
    results = []
    for entry in sorted(bf.listdir(data_dir)):
        full_path = bf.join(data_dir, entry)
        ext = entry.split(".")[-1]
        if "." in entry and ext.lower() in ["jpg", "jpeg", "png", "gif"]:
            results.append(full_path)
        elif bf.isdir(full_path):
            results.extend(_list_image_files_recursively(full_path))
    return results

Global seed set to 47


# How much background affect the lighting estimation from DECA?

In [2]:
# Load images of foreground/background/original

fgr_file = _list_image_files_recursively('./fgr/')
bgr_file = _list_image_files_recursively('./bgr/')
org_file = _list_image_files_recursively('./org/')
print(len(fgr_file), len(bgr_file), len(org_file))



file_dict = {}
for path in org_file:
    name = path.split('/')[-1]
    file_dict[name] = {'org':None, 'fgr':None}
    file_dict[name]['org'] = path
    file_dict[name]['fgr'] = [p for p in fgr_file if name.split('.')[0] in p][0]
    assert file_dict[name]['fgr'] != None
    assert file_dict[name]['org'] != None

# Check all images have paired.
tmp = copy.deepcopy(fgr_file)
for i in list(file_dict.keys()):
    i = i.split('.')[0]
    for idx, j in enumerate(tmp):
        if i in j:
            del(tmp[idx])
assert tmp == []


32 19 32


In [6]:
image_dict = file_dict.copy()
for name, v in file_dict.items():
    for k in ['org', 'fgr']:
        with bf.BlobFile(v[k], "rb") as f:
            pil_image = PIL.Image.open(f)
            pil_image.load()
            pil_image = pil_image.convert("RGB")
        image_dict[name][k] = pil_image

print(image_dict['60008.jpg'])


AttributeError: 'PixelAccess' object has no attribute 'decode'